### Necessary libraries

In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
from sklearn import linear_model
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sns
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
import tensorflow as tf
from keras.layers import Dense, Dropout, Activation, Flatten
from sklearn import metrics 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

### Preparing for classification

Loading the previously saved matrix containing feature vectors

In [ ]:
features = np.load('C:/User/gosia/SpeakerRecognition/data/features', allow_pickle=True)

Combine features into one vector for each recording

In [ ]:
all_features = []

for i in range(0, len(features)):
    all_features.append(np.concatenate((features[i][0], features[i][3], features[i][4], features[i][5]), axis=0))

Setting the labels

In [ ]:
labels = data['label']
np.unique(labels, return_counts=True)

In [ ]:
X = np.array(all_features)
y = np.array(labels)

### Dataset division

For the needs of the learning process with the use of the neural network, the set was divided into training (60%), test (20%) and validation (20%). 

In [ ]:
X_train, X_test, y_train, y_test  = train_test_split(X, y, test_size=0.2, random_state=70)
X_train, X_val, y_train, y_val  = train_test_split(X_train, y_train, test_size=0.25, random_state=500) # 0.25 x 0.8 = 0.2

Hot encoding y

In [ ]:
lb = LabelEncoder()
y_train = to_categorical(lb.fit_transform(y_train))
y_test = to_categorical(lb.fit_transform(y_test))
y_val = to_categorical(lb.fit_transform(y_val))

In [ ]:
print(X.shape)
print(y_test.shape)

In [ ]:
ss = StandardScaler()
X_train = ss.fit_transform(X_train)
X_val = ss.transform(X_val)
X_test = ss.transform(X_test)

### Network model

In [ ]:
model = Sequential()

model.add(Dense(64, input_shape=(196,), activation = 'relu'))
model.add(Dropout(0.5))

model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.2))  

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.2))  

model.add(Dense(34, activation = 'softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

EarlyStop = EarlyStopping(monitor='val_loss', 
                          patience=30,
                          verbose=1)

In [ ]:
print(model.summary())

### Network learning process

In [ ]:
history = model.fit(X_train, 
                    y_train, 
                    epochs=200, 
                    batch_size = 3,
                    validation_data=(X_val, y_val),  
                    steps_per_epoch = 100, 
                    callbacks = [EarlyStop], 
                    validation_steps = 20)

### Results

In [ ]:
# Check out train accuracy and validation accuracy over epochs.
train_accuracy = history.history['accuracy']
val_accuracy = history.history['val_accuracy']

# Set figure size.
plt.figure(figsize=(12, 8))

# Generate line plot of training, testing loss over epochs.
plt.plot(train_accuracy, label='Training Accuracy', color='#185fad')
plt.plot(val_accuracy, label='Validation Accuracy', color='orange')

# Set title
plt.title('Training and Validation Accuracy by Epoch', fontsize = 25)
plt.xlabel('Epoch', fontsize = 18)
plt.ylabel('Accuracy', fontsize = 18)
plt.xticks(range(0,200,10), range(0,200,10))
plt.legend(fontsize = 18);

In [ ]:
# Getting predictions from the test data
y_test_pred = model.predict(X_test)
y_test_pred = np.argmax(y_test_pred,axis=-1)

In [ ]:
y_test = np.argmax(y_test,axis=-1)

In [ ]:
false = 0
for i in range(0,len(y_test)):
  if y_test[i] != y_test_pred[i]:
    false+=1

# Checking model accuracy
acc = 1 - (false / len(y_test_pred))
print(acc)

In [ ]:
cm = confusion_matrix(y_test, y_test_pred)
print(cm)

In [ ]:
df_cm = pd.DataFrame(cm)
plt.figure(figsize=(12,10))
sns.heatmap(df_cm, annot=True, cmap = 'Blues')
plt.ylabel('True label')
plt.xlabel('Predicted label')

In [ ]:
print(classification_report(y_test, y_test_pred)) 